# RAG Post Retrieval Optimization - Reranking featuring Amazon SageMaker, Bedrock and llamaindex

In this tutorial, we showcase how to use a sub question query engine to tackle the problem of answering a complex query using multiple data sources.
It first breaks down the complex query into sub questions for each relevant data source, then gather all the intermediate reponses and synthesizes a final response.

- Vector Database (Faiss / local)
- LLM (Amazon Bedrock - Claude3 Sonnet)
- Embeddings Model (Bedrock Titan Text Embedding v2.0)
- Datasets ( Amazon SEC 10-k statements for year 2022 and 2023 )

In [1]:
!pip install llama-index
%pip install llama-index-llms-bedrock
%pip install llama-index-embeddings-bedrock
!pip uninstall pydantic -y
!pip install pydantic
%pip install sqlalchemy==2.0.21 --force-reinstall --quiet
%pip install llama-index-embeddings-instructor
%pip install llama-index-embeddings-huggingface

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Found existing installation: pydantic 2.8.2
Uninstalling pydantic-2.8.2:
  Successfully uninstalled pydantic-2.8.2
  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
Using cached pydantic-2.8.2-py3-none-any.whl (423 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 2.14.1 requires packaging>=22.0, but you have packaging 21.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from llama_index.embeddings.bedrock import BedrockEmbedding

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import Settings

In [4]:
import json
from typing import Sequence, List
from llama_index.core.settings import Settings
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding, Models

llm = Bedrock(model = "anthropic.claude-3-sonnet-20240229-v1:0")
embed_model = BedrockEmbedding(model = "amazon.titan-embed-text-v2:0")

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
import nest_asyncio
nest_asyncio.apply()

In [5]:
%pip install torch sentence-transformers
from llama_index.core.postprocessor import SentenceTransformerRerank
postprocessor = SentenceTransformerRerank(
    model="mixedbread-ai/mxbai-rerank-large-v1", top_n=3
)

Note: you may need to restart the kernel to use updated packages.


In [6]:
!mkdir -p 'data/amazon/'
!wget 'https://s2.q4cdn.com/299287126/files/doc_financials/2023/q4/c7c14359-36fa-40c3-b3ca-5bf7f3fa0b96.pdf' -O 'data/amazon/amazon_2023.pdf'
!wget 'https://s2.q4cdn.com/299287126/files/doc_financials/2022/q4/d2fde7ee-05f7-419d-9ce8-186de4c96e25.pdf' -O 'data/amazon/amazon_2022.pdf'

--2024-07-26 18:38:32--  https://s2.q4cdn.com/299287126/files/doc_financials/2023/q4/c7c14359-36fa-40c3-b3ca-5bf7f3fa0b96.pdf
Resolving s2.q4cdn.com (s2.q4cdn.com)... 68.70.205.4, 68.70.205.3, 68.70.205.1, ...
Connecting to s2.q4cdn.com (s2.q4cdn.com)|68.70.205.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 800598 (782K) [application/pdf]
Saving to: ‘data/amazon/amazon_2023.pdf’

100%[======================================>] 800,598     --.-K/s   in 0.06s   

2024-07-26 18:38:32 (13.8 MB/s) - ‘data/amazon/amazon_2023.pdf’ saved [800598/800598]

--2024-07-26 18:38:33--  https://s2.q4cdn.com/299287126/files/doc_financials/2022/q4/d2fde7ee-05f7-419d-9ce8-186de4c96e25.pdf
Resolving s2.q4cdn.com (s2.q4cdn.com)... 68.70.205.2, 68.70.205.1, 68.70.205.3, ...
Connecting to s2.q4cdn.com (s2.q4cdn.com)|68.70.205.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 712683 (696K) [application/pdf]
Saving to: ‘data/amazon/amazon_2022.pdf’

100%[===

In [7]:
# load data
amazon_secfiles = SimpleDirectoryReader(input_dir="./data/amazon/").load_data()

# build index
amazon_index = VectorStoreIndex.from_documents(
    amazon_secfiles,
    use_async=True,
)

# Retrieval - Sentence Transformers Rerank.

In [8]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
import pandas as pd
from IPython.display import display, HTML


pd.set_option("display.max_colwidth", -1)


def get_retrieved_nodes(
    query_str, vector_top_k=10, reranker_top_n=3, with_reranker=False
):
    query_bundle = QueryBundle(query_str)
    # configure retriever
    retriever = VectorIndexRetriever(
        index=amazon_index,
        similarity_top_k=vector_top_k,
    )
    retrieved_nodes = retriever.retrieve(query_bundle)

    if with_reranker:
        # configure reranker
        reranker = SentenceTransformerRerank(model="mixedbread-ai/mxbai-rerank-large-v1", top_n=3)
        retrieved_nodes = reranker.postprocess_nodes(
            retrieved_nodes, query_bundle
        )

    return retrieved_nodes


def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "")))


def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))

/tmp/ipykernel_1964/2169426670.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


In [9]:
new_nodes = get_retrieved_nodes(
    "What were key challenges faced by Amazon in year 2022?",
    vector_top_k=10,
    with_reranker=False,
)

In [10]:
visualize_retrieved_nodes(new_nodes)

,Score,Text
0,0.671755,"which has been critical to ourgrowth and success;•disruptions from natural or human-caused disasters (including public health crises) or extreme weather (including as a result of climate change),geopolitical events and security issues (including terrorist attacks, armed hostilities, and political conflicts, including those involving China), labor ortrade disputes (including restrictive governmental actions impacting us, our customers, and our third-party sellers and suppliers in China or otherforeign countries), and similar events; and•potential negative impacts of climate change, including: increased operating costs due to more frequent extreme weather events or climate-relatedchanges, such as rising temperatures and water scarcity; increased investment requirements associated with the transition to a low-carbon economy;decreased demand for our products and services as a result of changes in customer behavior; increased compliance costs due to more extensive andglobal regulations and third-party requirements; and reputational damage resulting from perceptions of our environmental impact.We Face Risks Related to Successfully Optimizing and Operating Our Fulfillment Network and Data CentersFailures to adequately predict customer demand and consumer spending patterns or otherwise optimize and operate our fulfillment network and datacenters successfully from time to time result in excess or insufficient fulfillment or data center capacity, service interruptions, increased costs, and impairmentcharges, any of which could materially harm our business. As we continue to add fulfillment and data center capability or add new businesses with differentrequirements, our fulfillment and data center networks become increasingly complex and operating them becomes more challenging. There can be no assurancethat we will be able to operate our networks effectively.10"
1,0.660597,"resources such as land, water, and energy, commodities like paper andpacking supplies and hardware products, and technology infrastructure products, including as a result of inflationary pressures;•constrained labor markets, which increase our payroll costs;•the extent to which operators of the networks between our customers and our stores successfully charge fees to grant our customers unimpaired andunconstrained access to our online services;•our ability to collect amounts owed to us when they become due;•the extent to which new and existing technologies, or industry trends, restrict online advertising or affect our ability to customize advertising orotherwise tailor our product and service offerings;•the extent to which use of our services is affected by spyware, viruses, phishing and other spam emails, denial of service attacks, data theft, computerintrusions, outages, and similar events;•the extent to which we fail to maintain our unique culture of innovation, customer obsession, and long-term thinking, which has been critical to ourgrowth and success;•disruptions from natural or human-caused disasters (including public health crises) or extreme weather (including as a result of climate change),geopolitical events and security issues (including terrorist attacks, armed hostilities, and political conflicts, including those involving China), labor ortrade disputes (including restrictive governmental actions impacting us, our customers, and our third-party sellers and suppliers in China or otherforeign countries), and similar events; and•potential negative impacts of climate change, including: increased operating costs due to more frequent extreme weather events or climate-relatedchanges, such as rising temperatures and water scarcity; increased investment requirements associated with the transition to a low-carbon economy;decreased demand for our products and services as a result of changes in customer behavior; increased compliance costs due to more extensive andglobal regulations and third-party requirements; and reputational damage result

In [11]:
new_nodes = get_retrieved_nodes(
    "What were key challenges faced by Amazon in year 2022?",
    vector_top_k=10,
    reranker_top_n=3,
    with_reranker=True,
)

In [12]:
visualize_retrieved_nodes(new_nodes)

,Score,Text
0,0.795661,"which has been critical to ourgrowth and success;•disruptions from natural or human-caused disasters (including public health crises) or extreme weather (including as a result of climate change),geopolitical events and security issues (including terrorist attacks, armed hostilities, and political conflicts, including those involving China), labor ortrade disputes (including restrictive governmental actions impacting us, our customers, and our third-party sellers and suppliers in China or otherforeign countries), and similar events; and•potential negative impacts of climate change, including: increased operating costs due to more frequent extreme weather events or climate-relatedchanges, such as rising temperatures and water scarcity; increased investment requirements associated with the transition to a low-carbon economy;decreased demand for our products and services as a result of changes in customer behavior; increased compliance costs due to more extensive andglobal regulations and third-party requirements; and reputational damage resulting from perceptions of our environmental impact.We Face Risks Related to Successfully Optimizing and Operating Our Fulfillment Network and Data CentersFailures to adequately predict customer demand and consumer spending patterns or otherwise optimize and operate our fulfillment network and datacenters successfully from time to time result in excess or insufficient fulfillment or data center capacity, service interruptions, increased costs, and impairmentcharges, any of which could materially harm our business. As we continue to add fulfillment and data center capability or add new businesses with differentrequirements, our fulfillment and data center networks become increasingly complex and operating them becomes more challenging. There can be no assurancethat we will be able to operate our networks effectively.10"
1,0.770071,"In addition, rising fuel, utility, and food costs, rising interest rates, and recessionary fears may impact customer demandand our ability to forecast consumer spending patterns. We also expect the current macroeconomic environment and enterprise customer cost optimizationefforts to impact our AWS revenue growth rates. We expect some or all of these factors to continue to impact our operations into Q1 2023.Net SalesNet sales include product and service sales. Product sales represent revenue from the sale of products and related shipping fees and digital media contentwhere we record revenue gross. Service sales primarily represent third-party seller fees, which includes commissions and any related fulfillment and shippingfees, AWS sales, advertising services, Amazon Prime membership fees, and certain digital content subscriptions. Net sales information is as follows (inmillions): Year Ended December 31, 2021 2022Net Sales:North America $ 279,833 $ 315,880 International 127,787 118,007 AWS 62,202 80,096 Consolidated $ 469,822 $ 513,983 Year-over-year Percentage Growth (Decline):North America 18 % 13 %International 22 (8)AWS 37 29 Consolidated 22 9 Year-over-year Percentage Growth, excluding the effect of foreign exchange rates:North America 18 % 13 %International 20 4 AWS 37 29 Consolidated 21 13 Net sales mix:North America 60 % 61 %International 27 23 AWS 13 16 Consolidated 100 % 100 %Sales increased 9% in 2022, compared to the prior year. Changes in foreign currency exchange rates reduced net sales by $15.5 billion in 2022. For adiscussion of the effect of foreign exchange rates on sales growth, see “Effect of Foreign Exchange Rates” below.North America sales increased 13% in 2022, compared to the prior year. The sales growth primarily reflects increased unit sales, including sales by third-party sellers, advertising sales, and subscription services."
2,0.726103,"They may secure better terms from vendors, adopt more aggressive pricing, and devote more resources to technology, infrastructure,fulfillment, and marketing.Competition continues to in

# Seeing reranking in action in final results

In [13]:
query_engine_naive = amazon_index.as_query_engine(similarity_top_k=10)

In [14]:
response = query_engine_naive.query(
    "What were key challenges faced by Amazon in year 2022?"
)

In [15]:
print(response)

Based on the context information provided, some of the key challenges faced by Amazon in 2022 included:

1. Inflationary pressures leading to increased costs for resources like land, water, energy, commodities, and technology infrastructure products.

2. Constrained labor markets resulting in higher payroll costs.

3. Disruptions from natural disasters, public health crises, extreme weather events (potentially due to climate change), geopolitical tensions, labor disputes, and similar events impacting operations.

4. Potential negative impacts of climate change, such as increased operating costs due to extreme weather, investment requirements for transitioning to a low-carbon economy, decreased demand due to changes in customer behavior, and reputational damage.

5. Challenges in optimizing and operating the fulfillment network and data centers, leading to excess or insufficient capacity, service interruptions, and increased costs.

6. Variability in retail demand, especially during pea

In [16]:
query_engine_rerank = amazon_index.as_query_engine(similarity_top_k=10, node_postprocessor =[postprocessor])

In [17]:
response = query_engine_rerank.query(
    "What were key challenges faced by Amazon in year 2022?"
)

In [18]:
print(response)

Based on the context information provided, some of the key challenges faced by Amazon in 2022 included:

1. Increased operating costs due to factors like rising fuel, utility, and food costs, as well as higher payroll costs from constrained labor markets.

2. Impacts from macroeconomic factors like rising inflation, recessionary fears, and enterprise customers' cost optimization efforts, which affected consumer spending patterns and AWS revenue growth rates.

3. Supply chain disruptions and fulfillment network challenges caused by factors like natural disasters, extreme weather events related to climate change, geopolitical conflicts, labor disputes, and similar events.

4. Difficulties in optimizing inventory management and staffing levels across the fulfillment network, leading to higher shipping costs and operational inefficiencies.

5. Dependence on a limited number of shipping companies, whose performance issues or capacity constraints could negatively impact order fulfillment and

# With LLMRerank Module

In [61]:
from llama_index.core.postprocessor import LLMRerank

In [62]:
llm = Bedrock(model = "anthropic.claude-3-haiku-20240307-v1:0")

In [63]:
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

In [66]:
#from llama_index.core import ServiceContext
#service_context = ServiceContext.from_defaults(llm=Settings.llm, 
 #                                              embed_model=Settings.embed_model, 
#                                               chunk_size=Settings.chunk_size,
#                                               chunk_overlap=Settings.chunk_overlap,
#                                            )

In [67]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
import pandas as pd
from IPython.display import display, HTML


pd.set_option("display.max_colwidth", -1)


def get_retrieved_nodes(
    query_str, vector_top_k=10, reranker_top_n=3, with_reranker=False
):
    query_bundle = QueryBundle(query_str)
    # configure retriever
    retriever = VectorIndexRetriever(
        index=amazon_index,
        similarity_top_k=vector_top_k,
    )
    retrieved_nodes = retriever.retrieve(query_bundle)

    if with_reranker:
        # configure reranker
        reranker = LLMRerank(choice_batch_size=3, top_n=reranker_top_n)
        retrieved_nodes = reranker.postprocess_nodes(
            retrieved_nodes, query_bundle
        )

    return retrieved_nodes


def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "")))


def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))

/tmp/ipykernel_1964/580038862.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


In [68]:
new_nodes = get_retrieved_nodes(
    "What were key challenges faced by Amazon in year 2022?",
    vector_top_k=5,
    with_reranker=False,
)

In [69]:
visualize_retrieved_nodes(new_nodes)

,Score,Text
0,0.671755,"which has been critical to ourgrowth and success;•disruptions from natural or human-caused disasters (including public health crises) or extreme weather (including as a result of climate change),geopolitical events and security issues (including terrorist attacks, armed hostilities, and political conflicts, including those involving China), labor ortrade disputes (including restrictive governmental actions impacting us, our customers, and our third-party sellers and suppliers in China or otherforeign countries), and similar events; and•potential negative impacts of climate change, including: increased operating costs due to more frequent extreme weather events or climate-relatedchanges, such as rising temperatures and water scarcity; increased investment requirements associated with the transition to a low-carbon economy;decreased demand for our products and services as a result of changes in customer behavior; increased compliance costs due to more extensive andglobal regulations and third-party requirements; and reputational damage resulting from perceptions of our environmental impact.We Face Risks Related to Successfully Optimizing and Operating Our Fulfillment Network and Data CentersFailures to adequately predict customer demand and consumer spending patterns or otherwise optimize and operate our fulfillment network and datacenters successfully from time to time result in excess or insufficient fulfillment or data center capacity, service interruptions, increased costs, and impairmentcharges, any of which could materially harm our business. As we continue to add fulfillment and data center capability or add new businesses with differentrequirements, our fulfillment and data center networks become increasingly complex and operating them becomes more challenging. There can be no assurancethat we will be able to operate our networks effectively.10"
1,0.660597,"resources such as land, water, and energy, commodities like paper andpacking supplies and hardware products, and technology infrastructure products, including as a result of inflationary pressures;•constrained labor markets, which increase our payroll costs;•the extent to which operators of the networks between our customers and our stores successfully charge fees to grant our customers unimpaired andunconstrained access to our online services;•our ability to collect amounts owed to us when they become due;•the extent to which new and existing technologies, or industry trends, restrict online advertising or affect our ability to customize advertising orotherwise tailor our product and service offerings;•the extent to which use of our services is affected by spyware, viruses, phishing and other spam emails, denial of service attacks, data theft, computerintrusions, outages, and similar events;•the extent to which we fail to maintain our unique culture of innovation, customer obsession, and long-term thinking, which has been critical to ourgrowth and success;•disruptions from natural or human-caused disasters (including public health crises) or extreme weather (including as a result of climate change),geopolitical events and security issues (including terrorist attacks, armed hostilities, and political conflicts, including those involving China), labor ortrade disputes (including restrictive governmental actions impacting us, our customers, and our third-party sellers and suppliers in China or otherforeign countries), and similar events; and•potential negative impacts of climate change, including: increased operating costs due to more frequent extreme weather events or climate-relatedchanges, such as rising temperatures and water scarcity; increased investment requirements associated with the transition to a low-carbon economy;decreased demand for our products and services as a result of changes in customer behavior; increased compliance costs due to more extensive andglobal regulations and third-party requirements; and reputational damage result

In [72]:
new_nodes = get_retrieved_nodes(
    "What were key challenges faced by Amazon in year 2022?",
    vector_top_k=10,
    reranker_top_n=3,
    with_reranker=True,
)

IndexError: list index out of range

In [ ]:
visualize_retrieved_nodes(new_nodes)